In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

import weightedstats as ws 
import wquantiles as wq

def wmedian(df, column_name, weights_name='WPFINWGT'):
    
    df = df.dropna(subset=[column_name,weights_name ])
    
    return ws.weighted_median( df[column_name], weights=df[weights_name])
    
def wmedian2(df, column_name, weights_name='WPFINWGT'):
    
    df = df.dropna(subset=[column_name,weights_name ])
    
    return wq.median( df[column_name], df[weights_name])
    
def wmean(df, column_name, weights_name='WPFINWGT'):
    """Calculate the weighted mean of a list."""

    w = df[weights_name]/df[weights_name].sum()
     
    return (df[column_name]*w).sum()



In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Survey of Income and Program Participation
`census.gov-sipp-1.1.1` Last Update: 2020-12-17T19:05:20

_SIPP collects data and measures change for many topics including: economic well-being, family dynamics, education, assets, health insurance, childcare, and food security._


## Documentation Links

* [Program home page](https://www.census.gov/programs-surveys/sipp.html) 
* [2018 data page](https://www.census.gov/programs-surveys/sipp/data/datasets/2018-data/2018.html) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_dta.zip#pu2018.dta">pu2018_dta</a></strong>. 2018 SIPP primary data, stata format</li><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/rw2018_sasdata.zip">rw2018_dta</a></strong>. 2018 SIPP replicate weights, stata format</li><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_csv.zip">pu2018_csv</a></strong>. 2018 SIPP primary data, CSV format</li><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/rw2018_csv.zip">rw2018_csv</a></strong>. 2018 SIPP replicate weights, CSV format</li><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_schema.json">pu2018_schema</a></strong>. 2018 Primary data schema</li><li> <strong><a href="https://www2.census.gov/programs-surveys/sipp/data/datasets/2018/rw2018_schema.json">rw2018_schema</a></strong>. 2018 Replicate weights schema</li><ul>

In [3]:
'''
The following code is an example of reading pipe-delimited Survey of Income and Program Participation (SIPP) 
    data into a Pandas dataframe. Specifically, this code loads and merges the primary dataset and the 
    calendar-year replicate weights (as opposed to the longitudinal replicate weights) in preparation for 
    analysis.
This code is written for Python 3, and requires version 0.24 or higher of the Pandas module. 
Note the use of 'usecols' in the first pd.read_csv statement. Most machines do not have enough memory to read
    the entire SIPP file into memory. Use 'usecols' to read in only the columns you are interested in. If you
    still encounter an out-of-memory error, either select less columns, or consider using the Dask module.
Run this code in the same directory as the data.
This code was written by Adam Smith. Please report errors to <census.sipp@census.gov>.
'''

#Read in the primary data file schema to get data-type information for
#    each variable.
rd_schema = pd.read_json(pkg.reference('pu2018_schema').resolved_url.get_resource().read())

#Read in the replicate weight data file schema to get data-type information 
#    for each variable.
rw_schema = pd.read_json(pkg.reference('rw2018_schema').resolved_url.get_resource().read())


#Define Pandas data types based on the schema data-type information for both schema dataframes
rd_schema['dtype'] = ['Int64' if x == 'integer' \
            else 'object' if x == 'string' \
            else 'Float64' if x == 'float' \
            else 'ERROR' \
            for x in rd_schema['dtype']]

rw_schema['dtype'] = ['Int64' if x == 'integer' \
            else 'object' if x == 'string' \
            else 'Float64' if x == 'float' \
            else 'ERROR' \
            for x in rw_schema['dtype']]

In [4]:
df = pkg.reference('pu2018_dta').dataframe()

!!!! file:///Users/eric/Library/Application%20Support/metapack/www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_dta.zip_d/pu2018.dta extant
!!!! file:///Users/eric/Library/Application%20Support/metapack/www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_dta.zip_d/pu2018.dta extant


In [24]:

# Get labels for columns with 
# dict(zip(rd_schema['name'], rd_schema['label']))

col_label_map = dict(zip(rd_schema['name'], rd_schema['label']))
dtype_map = dict(zip(rd_schema['name'], rd_schema['dtype']))

use_cols = [
    'ssuid', # Sample unit identifier. This identifier is created by scrambling together PSU, Sequence #1, Sequence #2, and the Frame Indicator for a case. It may be used in matching sample units from different waves.
    'pnum', # Person number
    'monthcode', # Value of reference month
    'eresidenceid', # This field stores a unique six-digit identifier for residence addresses.
    'erelrpe', # Household relationship (detailed categories)
    'spanel', # Panel year
    'swave', # Wave number of interview
    'wpfinwgt', # Final person weight
    'THHLDSTATUS',
    'rin_univ', # Monthly indicator that respondent is in survey frame universe
    'esex', # Sex of this person
    'tage', # Age as of last birthday
    'tage_ehc', # Monthly age during the reference year.
    'erace', # What race(s) does ... consider herself/himself to be?
    'trace', # What race(s) does ... consider herself/himself to be? (detailed categories)
    'eorigin', # Is ... Spanish, Hispanic, or Latino?
    'eeduc', # What is the highest level of school ... completed or the highest degree received by December of (reference year)?
    'ems', # Is ... currently married, widowed, divorced, separated, or never married?
    'ebornus', # Where was ... born?
    'eawbsafe', # Is ... neighborhood safe from crime?
    'edinrpar', # How many days in a typical week did reference parent eat dinner with child/children?
    'eexpsch', # Child/children ever been suspended or expelled from school?
    'rexpsch', # Child was expelled
    'egifted', # Child/children in gifted class/classes.
    'rlesson', # Child takes lessons
    'rgetby', # Child does just enough to get by
    'efood1', # The food you bought did not last?
    'efood6', # In 2017, were you ever hungry but didn't eat because there wasn't enough money for food?
    'rbreak_chld', # Did child usually get the school breakfast that his/her school provided?
    'rlunch_chld', # Did child usually get the school lunch that his/her school provided?
    'efood_mnyn', # Did ... receive food assistance (not SNAP) this month (1-12)?
    'efood_sr1yn', # Did ... receive assistance from a government social service agency?
    'rdinrpar', # Number of times reference parent had dinner with child 0-17.
    'rdinrop', # Number of times other parent had dinner with child 0-17.
    'rsnap_mnyn', # Received SNAP benefits this month
    'rsnap_yryn', # Received SNAP benefits in at least one month of the reference period
    'ttanf_amt', # Value of the TANF benefits received this month
    'empf', # Do you have children with more than one partner?
    'tage_fb', # Age at first birth
    'tnum_cbu', # Number of child bearing unions
    'exmar', # Number of times married
    'tprvlvqrt', # Type of living quarters for the residence.
    'tpearn', # Sum of earnings and profits/losses from all jobs, varying with the number of days in the month (comparable to RPEARN in 1996, 2001, and 2014 SIPP panels).
    'tmwkhrs', # Average number of hours worked per week at all jobs held during the reference month.
    'tptotinc', # Sum of personal monthly earnings and income for people age 15 and older, as well as children under age 15 who received SSI payments
    'tptrninc', # Sum of monthly income received from means-tested transfer programs (including SSI, TANF, GA, and the Veterans Pension program)
    'thnetworth', # Household-level net worth [this is household-level data, therefore this value is copied to every member of the household].
    'thval_ast', # Household-level sum of all asset values (TVAL_AST) [this is household-level data, therefore this value is copied to every member of the household].
    'thval_home', # Household-level sum of value of primary residence (TVAL_HOME) [this is household-level data, therefore this value is copied to every member of the household].
    'thdebt_ast', # Household-level sum of all debt (TDEBT_AST) [this is household-level data, therefore this value is copied to every member of the household].
    'tdebt_ast', # Person-level sum of all debt (TDEBT_SEC, TDEBT_USEC).
    'thinc_stmf', # Household-level sum of income earned over the reference period from stocks and mutual funds (TINC_STMF) [this is household-level data, therefore this value is copied to every member of the household].
    'eoeddebt', # Owed any money for student loans or educational expenses in own name only during the reference period.
    'toeddebtval', # Amount of student loans or educational expenses owed in own name only as of the last day of the reference period.
    'ejseddebt', # Owed any money for student loans or educational expenses jointly with a spouse or civil union partner during the reference period.
    'eprocert', # Has...earned a professional certification or license?
    'ejb1_wshmwrk', # Were there any days when ... worked only from home?
    'ejb1_wsjob', # What is the best description of ... work schedule?
    'tjb1_pvtime', # What is ... one-way travel to work in minutes?
    'rfamkind', # Kind of family
    'rfamref', # Person number of the family reference person
    'rhnumper', # Number of persons in household this month
    'rfrelu18', # Number of persons in family under 18 years
    'rhnum65over', # Number of persons in household 65 years and over this month
    'rhnum65ovrt2', # Number of persons in household 65 years and over this month (with Type 2 persons)
    'rfpersons', # Number of persons in family
    'rfrelu18', # Number of persons in family under 18 years
    'rpar1sex', # Parent 1 sex at interview month
    'rpar2sex', # Parent 2 sex at interview month
    'ems', # Is ... currently married, widowed, divorced, separated, or never married?
    'epnspouse', # Person number of spouse
    'epar1typ', # Type of relationship to parent 1
    'epar2typ', # Type of relationship to parent 2
    'epnpar1', # Person number of parent 1
    'epnpar2', # Person number of parent 2
    
]

if False:
    for c in use_cols:
        print(f"    '{c.lower()}', # {col_label_map[c]}")


In [26]:
%%time
df_data = pkg.reference('pu2018_csv').read_csv(
    sep='|',
    usecols = [ c.upper() for c in use_cols],
    dtype={c:dtype_map[c.upper()] for c in use_cols })

!!!! file:///Users/eric/Library/Application%20Support/metapack/www2.census.gov/programs-surveys/sipp/data/datasets/2018/pu2018_csv.zip_d/pu2018.csv extant
CPU times: user 1min 3s, sys: 1min 35s, total: 2min 38s
Wall time: 3min 30s


In [8]:
df_data.columns = [c.lower() for c in df_data.columns]

if False:
    df_rw_2 = pkg.reference('rw2018_csv').read_csv(sep='|')

    #check these estimates against the validation xls file to help ensure that the data
    #    were read in correctly. Note that the validation xls files do not include all variables
    print('REPWT100 mean:' + str(df_rw.REPWGT100.mean()))

    #Merge data and replicate weights on SSUID, PNUM, MONTHCODE
    df = df_data.merge(df_rw, on=['ssuid','pnum','monthcore'])
else:
    df = df_data
    
    
df = df.rename(columns={'swave_x':'swave', 'spane;_x':'spanel'})

def raceeth(r):
    if r.eorigin == 1:
        return 'hisp'
    elif r.erace == 1:
        return 'white'
    elif r.erace == 2:
        return 'black'
    elif r.erace == 3:
        return 'asian'
    else:
        return 'other'

df['raceeth'] = df.apply(raceeth,axis=1)
df['dummy'] = 1 # For counting

In [9]:
dfs = df.sample(1_000_000, replace=True, weights=df.wpfinwgt)

In [10]:
# Commute Time
t = dfs.groupby(['raceeth']).tjb1_pvtime.describe()
t


,count,mean,std,min,25%,50%,75%,max
raceeth,,,,,,,,
asian,26329.0,26.490144,16.300586,1.0,15.0,24.0,30.0,68.0
black,52486.0,25.471993,15.940003,1.0,15.0,24.0,30.0,68.0
hisp,77164.0,25.087204,16.041400,1.0,15.0,24.0,30.0,68.0
other,11164.0,23.746865,16.701501,1.0,10.0,20.0,30.0,68.0
white,280586.0,23.261310,15.920373,1.0,10.0,20.0,30.0,68.0


In [11]:
mp.__file__

'/Users/eric/proj/metatab-dev/metapack-dev/metapack/src/metapack/__init__.py'

In [13]:
df_data.groupby('pnum').ssuid.count()

pnum
101    314580
102    219448
103    112719
104     67385
105     29511
106     11569
107      4573
108      1860
109       870
110       323
111       192
112        84
113        54
114        18
Name: ssuid, dtype: int64

In [ ]:
t = df[(df.MONTHCODE==1) & (df.raceeth.isin(['white','black']))]
(t.groupby('raceeth').WPFINWGT.sum()/t.WPFINWGT.sum())

In [ ]:
dfh = df.groupby(['SSUID']).agg({'ESEX':'first','raceeth':'first','WPFINWGT': 'first','TPTOTINC': 'sum', 'THNETWORTH' : 'last'})
dfh.groupby('raceeth').apply(wmedian, 'TPTOTINC')

In [ ]:

dfh.groupby('raceeth').apply(wmedian, 'THNETWORTH')

In [ ]:
def wquantile(x,q):           
    xsort = x.sort_values(x.columns[0])
    xsort['index'] = range(len(x))
    p = q * x[x.columns[1]].sum()
    pop = float(xsort[xsort.columns[1]][xsort['index']==0])
    i = 0
    while pop < p:
        pop = pop + float(xsort[xsort.columns[1]][xsort['index']==i+1])
        i = i + 1
    return xsort[xsort.columns[0]][xsort['index']==i]

In [ ]:
wmedian(dfh, 'TPTOTINC')

In [ ]:
q = .5

# Simple weighted quantiles

# Sort the dataframe by the value you want to compute quantiles on
dfh.sort_values('TPTOTINC', inplace=True)

# Compute the cumullate sum of the weights, then divide by the sum of the weights to 
# get a cumulative proportion (0,1)
dfh['TPTOTINC_q'] = dfh.WPFINWGT.cumsum()/dfh.WPFINWGT.sum()

# Now a quantile value is the smallest value in the set of rows that have a quantile proportion
# larger than the desired quantile level. For instance, the 50% percentile: 
dfh[dfh.TPTOTINC_q>.5].TPTOTINC.min()

# or 

dfh[dfh.TPTOTINC_q>.5].TPTOTINC.iloc[0]




In [ ]:
t = dfh[dfh.TPTOTINC_q < .9]
t = t.groupby([np.floor(dfh.TPTOTINC_q*10), 'raceeth']).THNETWORTH.mean().unstack()
t.plot()

In [ ]:
%timeit t[t.TPTOTINC_q>.5].TPTOTINC.min()

In [ ]:
%timeit t[t.TPTOTINC_q>.5].TPTOTINC.iloc[0]

In [ ]:
dfh[dfh.TPTOTINC_q>.5].TPTOTINC.min()

In [ ]:
dfh[dfh.TPTOTINC_q>.5].TPTOTINC.argmin

In [ ]:
dfh[dfh.TPTOTINC_q.round(2)*100==50].mean()

In [ ]:
rd_schema

In [ ]:
dtype_map = rd_schema.set_index('name')['dtype'].to_dict()

In [ ]:
df.groupby(['SSUID','PNUM', 'EFOOD_MNYN']).dummy.count().unstack()[1].fillna(0).value_counts()

In [14]:
"""
    1. Married couple
    2. Female, no spouse present
    3. Male, no spouse present 
"""
df.rfamkind.value_counts()

1    452985
2    125985
3     24667
Name: rfamkind, dtype: Int64

In [20]:
df.groupby(['rhnumper','monthcode']).dummy.count().unstack()

monthcode,1,2,3,4,5,6,7,8,9,10,11,12
rhnumper,,,,,,,,,,,,
1,9453,9403,9360,9310,9232,9145,9027,8844,8718,8601,8495,8369
2,17912,17890,17874,17846,17786,17704,17696,17686,17716,17674,17662,17642
3,11166,11166,11175,11196,11196,11229,11235,11277,11259,11268,11280,11265
4,12136,12164,12204,12236,12336,12372,12432,12504,12580,12620,12704,12800
5,6945,7010,7030,7050,7125,7200,7205,7250,7260,7305,7305,7355
6,3162,3162,3198,3222,3246,3324,3360,3390,3408,3462,3504,3510
7,1393,1428,1428,1456,1442,1449,1498,1512,1547,1561,1575,1596
8,592,592,592,592,608,600,600,624,640,680,696,712
9,261,279,270,270,261,270,288,297,324,333,351,369
